<h1>Matt's eCFR API Demonstrator</h1>
<p>An intro to Jupyter Notebooks</p>
<p><i>Teach yourself Python using the eCFR API</i></p>

In [41]:
# Do my imports - datetime for date operations, pandas for data analysis, requests for HTTPS REST API requests, json for, well, javascript object notation, duh
import datetime as dt
import pandas as pd
import requests as rq
import json as json

# Set my constants
base_url = "https://www.ecfr.gov/api/"
titles_url = base_url + "versioner/v1/titles.json"
agencies_url = base_url + "admin/v1/agencies.json"
data_path = "../data/"

# Define my eCFR API wrapper class
class eCFR_API_Wrapper:
    """
    A wrapper for the eCFR API to fetch and process data.
    """

    def __init__(self, base_url):
        self.base_url = base_url

    def fetch_data(endpoint):
        """
        Fetch data from the eCFR API.
        """
        try:
            response = rq.get(f"{endpoint}")
            if response.status_code == 200:
                return response.json()
            else:
                response.raise_for_status()
        except: 
            print(f"Error fetching data from {endpoint}")
            return None

In [ ]:
class eCFR_HouseKeeping:

    def __init__(self, data_path):
        self.data_path = data_path

    def refresh_titles(self):
        """
        Check the titles.json file for freshness and update if necessary.
        """
        statusmsg = "Titles check completed"
        # Get the current date and calculate the date 30 days ago
        thirtydaysback = dt.datetime.now() - dt.timedelta(days=30)
        
        try:
            with open(self.data_path + 'titles.json', 'r') as mytitlesfile:
                titles_json = json.load(mytitlesfile)
        except FileNotFoundError:
            titles_json = None
            statusmsg = "No titles.json found, need to fetch data"

        if titles_json:
            # We have some data, let's check it
            titles_dict = titles_json.get('titles', [])
            if not titles_dict:
                statusmsg = "No titles found in JSON"
                return statusmsg
            
            lastbestdate = None
            key_to_find = "up_to_date_as_of"
            up_to_date_values = [t[key_to_find] for t in titles_dict if key_to_find in t]
            lastbestdate = max(list(filter(None, up_to_date_values)))
            
            if lastbestdate > str(thirtydaysback):
                statusmsg += f" with good data as of {lastbestdate}"
            else:     
                statusmsg += f" with stale data as of {lastbestdate}. We need to refresh"
                # call refresh API here
                new_titles = eCFR_API_Wrapper.fetch_data(titles_url)
                if new_titles:
                    # we got new data - update the titles_json
                    titles_json = new_titles
                    # and write it to the file
                    with open(self.data_path + 'titles.json', 'w') as mytitlesfile:
                        json.dump(titles_json, mytitlesfile)
                    statusmsg += " and refreshed titles.json"
        else:
            # failed to load useful data - darn.    
            statusmsg += " but data is no good"
    
        return statusmsg

<h2>Put classes and constants above<h2>
<h3>Put implementation below</h3>

In [43]:

# initialize variables
titles_json= {'titles':[]}  #the titles_json dictionary to contain data
statusmsg = "initialized"  #use this to track what's going on 
lastbestdate = "1776-07-04"  #when was the eCFR last updated?
today = dt.date.today() #what day is it?

# test successful initialization
assert len(titles_json)>0
assert len(statusmsg)>0
assert str(today)>lastbestdate

print("Welcome to the MRWeCFR with base_url of "+base_url)

# Psuedocode
# 1. Let's see if we have a list of recent titles (less than 30 days old) in our titles.json file
# 2. If we do, let's see if we have the full and amendments of similar age, else, try and refresh titles.json 
# 3. If we are good with source data, skip to analysis, else, try and refresh the eCFR json details 
# 4. Analysis - let's load some basic semantic checks (wordcount, reading level) and some change trackers (count of changes, frequency over time)

#let's check the titles.json file for freshness
ecfr_housekeeping = eCFR_HouseKeeping(data_path)
statusmsg = ecfr_housekeeping.refresh_titles()
print("finished titles check with status of "+statusmsg)





Welcome to the MRWeCFR with base_url of https://www.ecfr.gov/api/
finished titles check with status of Titles check completed with good data as of 2025-07-08
finished titles check with status of Titles check completed with good data as of 2025-07-08
